In [8]:
import re
from io import StringIO

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from scipy.stats import kurtosis, normaltest, skew

log_path = (
    "/Users/armaan/Desktop/imc-prosperity-quant-nyc/submissions/2024-04-20_04-47-05.log"
)


def read_file_sections(filepath):
    section_delimiters = ["Sandbox logs:", "Activities log:", "Trade History:"]
    current_section = None
    data = {key: [] for key in section_delimiters}

    with open(filepath, "r") as file:
        for line in file:
            line = line.strip()
            if line in section_delimiters:
                current_section = line
                continue
            if current_section:
                data[current_section].append(line)

    return data


def extract_trades(text):
    # Regex to find all blocks enclosed in curly braces
    trade_pattern = r"\{[^{}]*\}"
    # Regex to extract key-value pairs within curly braces
    kv_pattern = r"\"(\w+)\":\s*(\"[^\"]*\"|\d+)"

    trades = []
    matches = re.findall(trade_pattern, text)
    for match in matches:
        trade_data = {}
        for key, value in re.findall(kv_pattern, match):
            if value.startswith('"') and value.endswith('"'):
                value = value.strip('"')
            else:
                value = int(value)
            trade_data[key] = value
        trades.append(trade_data)
    return trades

In [9]:
data_bottle_paths = [
    "/Users/armaan/Desktop/imc-prosperity-quant-nyc/data_bottles/round-5/trades_round_1_day_-1_wn.csv",
    "/Users/armaan/Desktop/imc-prosperity-quant-nyc/data_bottles/round-5/trades_round_1_day_-2_wn.csv",
    "/Users/armaan/Desktop/imc-prosperity-quant-nyc/data_bottles/round-5/trades_round_1_day_0_wn.csv",
    "/Users/armaan/Desktop/imc-prosperity-quant-nyc/data_bottles/round-5/trades_round_3_day_0_wn.csv",
    "/Users/armaan/Desktop/imc-prosperity-quant-nyc/data_bottles/round-5/trades_round_3_day_1_wn.csv",
    "/Users/armaan/Desktop/imc-prosperity-quant-nyc/data_bottles/round-5/trades_round_3_day_2_wn.csv",
    "/Users/armaan/Desktop/imc-prosperity-quant-nyc/data_bottles/round-5/trades_round_4_day_1_wn.csv",
    "/Users/armaan/Desktop/imc-prosperity-quant-nyc/data_bottles/round-5/trades_round_4_day_2_wn.csv",
    "/Users/armaan/Desktop/imc-prosperity-quant-nyc/data_bottles/round-5/trades_round_4_day_3_wn.csv",
]

r5_live_submission_log = (
    "/Users/armaan/Desktop/imc-prosperity-quant-nyc/submissions/2024-04-24_21-39-19.log"
)

sections = read_file_sections(r5_live_submission_log)
activities_data = "\n".join(sections["Activities log:"])
activities_df = pd.read_csv(StringIO(activities_data), delimiter=";")
trade_history_text = "\n".join(sections["Trade History:"])
trades = extract_trades(trade_history_text)
trade_history_df = pd.DataFrame(trades)


1. Vinnie: Net P&L for ROSES = 3466.0
2. Vinnie: Net P&L for STRAWBERRIES = 5104.5
---
1. Vladimir: Net P&L for CHOCOLATE = 1789.5 (ONLY SELLS)
2. Vladimir: Net P&L for GIFT_BASKET = 5315.0
3. Vladimir: Net P&L for ROSES = 1861.5 (ONLY SELLS)
---
1. Rhianna: Net P&L for COCONUT_COUPON = 318.0
2. Rhianna: Net P&L for GIFT_BASKET = 1987.0
---
1. Raj: Net P&L for COCONUT = 281.0
---
1. Ruby: Net P&L for COCONUT_COUPON = 1161.5
2. Ruby: Net P&L for GIFT_BASKET = 3257.0

In [10]:
TRADER = "Vladimir"

In [11]:
filtered_trades = trade_history_df[(trade_history_df["buyer"] == TRADER) | (trade_history_df["seller"] == TRADER)]
SYMBOLS_TRADED = sorted(list(filtered_trades['symbol'].unique()))


In [12]:
SYMBOLS_TRADED


['AMETHYSTS',
 'CHOCOLATE',
 'COCONUT_COUPON',
 'GIFT_BASKET',
 'ORCHIDS',
 'ROSES',
 'STARFRUIT',
 'STRAWBERRIES']

In [13]:
for PRODUCT in SYMBOLS_TRADED:
    sections = read_file_sections(r5_live_submission_log)
    activities_data = "\n".join(sections["Activities log:"])
    activities_df = pd.read_csv(StringIO(activities_data), delimiter=";")
    trade_history_text = "\n".join(sections["Trade History:"])
    trades = extract_trades(trade_history_text)
    trade_history_df = pd.DataFrame(trades)

    mid_price_df = activities_df[["timestamp", "product", "mid_price"]]

    # Pivot the DataFrame to normalize it by timestamp with multiple values
    mid_price_df = mid_price_df.pivot_table(
        index="timestamp",
        columns="product",
        values=["mid_price"],
        # aggfunc="mean",
    ).reset_index()

    # Flatten the MultiIndex in columns and rename appropriately
    mid_price_df.columns = [
        f"{col[1].lower()}_{col[0]}" if col[1] else col[0] for col in mid_price_df.columns
    ]

    product_mid_price_column = f"{PRODUCT.lower()}_mid_price"
    assert product_mid_price_column in mid_price_df.columns

    trade_history_df = trade_history_df[
        ((trade_history_df["buyer"] == TRADER) | (trade_history_df["seller"] == TRADER))
        & (trade_history_df["symbol"] == PRODUCT)
    ].copy()

    # Computing signed quantity
    trade_history_df["signed_quantity"] = trade_history_df.apply(
        lambda row: (
            row["quantity"]
            if row["buyer"] == TRADER
            else -row["quantity"] if row["seller"] == TRADER else 0
        ),
        axis=1,
    )

    # Creating pivot table
    trade_history_pivot_df = trade_history_df.pivot_table(
        index="timestamp", columns="symbol", values="signed_quantity", aggfunc="sum"
    )

    trade_history_pivot_df.columns = [
        f"{col}_trades_made" for col in trade_history_pivot_df.columns
    ]

    products_trades_made_column = f"{PRODUCT}_trades_made"

    assert products_trades_made_column in trade_history_pivot_df

    normalized_df = mid_price_df[["timestamp", product_mid_price_column]].copy()
    merged_df = pd.merge(normalized_df, trade_history_pivot_df, on="timestamp", how='left')

    # Creating the plot with Plotly
    fig = go.Figure()

    # Add mid price trace
    fig.add_trace(
        go.Scatter(
            x=merged_df["timestamp"],
            y=merged_df[product_mid_price_column],
            mode="lines",
            name="Mid Price",
        )
    )

    # Add trade arrows
    trade_col = f"{PRODUCT}_trades_made"
    if trade_col in merged_df.columns:
        for _, row in merged_df.iterrows():
            if row[trade_col] > 0:  # Buy
                fig.add_trace(
                    go.Scatter(
                        x=[row["timestamp"]],
                        y=[row[product_mid_price_column]],
                        mode="markers",
                        marker_symbol="triangle-up",
                        marker_color="green",
                        marker_size=10,
                        name="Buy",
                    )
                )
            elif row[trade_col] < 0:  # Sell
                fig.add_trace(
                    go.Scatter(
                        x=[row["timestamp"]],
                        y=[row[product_mid_price_column]],
                        mode="markers",
                        marker_symbol="triangle-down",
                        marker_color="red",
                        marker_size=10,
                        name="Sell",
                    )
                )

    fig.update_layout(
        title=f"Time Series of Mid Price and Trades Made for {PRODUCT} by {TRADER}",
        xaxis_title="Timestamp",
        yaxis_title="Price / Trades Made",
        legend_title="Legend",
        font=dict(family="Courier New, monospace", size=12, color="RebeccaPurple"),
    )


    df = trade_history_df
    df_filtered = df[df["symbol"] == PRODUCT]
    # Filter where trader is either buyer or seller
    df_trader = df_filtered[
        (df_filtered["buyer"] == TRADER) | (df_filtered["seller"] == TRADER)
    ]
    df_trader["cum_qty"] = df_trader["signed_quantity"].cumsum()
    df_trader["cash_value"] = (df_trader["price"] * -df_trader["signed_quantity"]).cumsum()
    final_cash_value = df_trader.iloc[-1]["cash_value"]
    final_qty = df_trader.iloc[-1]["cum_qty"]
    # liq_val = df_trader.iloc[-1]["price"]
    liq_val = activities_df[activities_df["product"] == PRODUCT].iloc[-1]["mid_price"]
    net_pnl = final_cash_value + (final_qty * liq_val)
    print(f"{TRADER}: Net P&L for {PRODUCT} = {net_pnl}")

    # Show the figure
    fig.show()

Vladimir: Net P&L for AMETHYSTS = 272.0


Vladimir: Net P&L for CHOCOLATE = 1789.5


Vladimir: Net P&L for COCONUT_COUPON = -13362.0


Vladimir: Net P&L for GIFT_BASKET = -13995.0


Vladimir: Net P&L for ORCHIDS = 3480.0


Vladimir: Net P&L for ROSES = 1862.0


Vladimir: Net P&L for STARFRUIT = 282.0


Vladimir: Net P&L for STRAWBERRIES = -4872.0


In [14]:
for PRODUCT in SYMBOLS_TRADED:
    sections = read_file_sections(r5_live_submission_log)
    activities_data = "\n".join(sections["Activities log:"])
    activities_df = pd.read_csv(StringIO(activities_data), delimiter=";")
    trade_history_text = "\n".join(sections["Trade History:"])
    trades = extract_trades(trade_history_text)
    trade_history_df = pd.DataFrame(trades)

    mid_price_df = activities_df[["timestamp", "product", "mid_price"]]

    # Pivot the DataFrame to normalize it by timestamp with multiple values
    mid_price_df = mid_price_df.pivot_table(
        index="timestamp",
        columns="product",
        values=["mid_price"],
        # aggfunc="mean",
    ).reset_index()

    # Flatten the MultiIndex in columns and rename appropriately
    mid_price_df.columns = [
        f"{col[1].lower()}_{col[0]}" if col[1] else col[0] for col in mid_price_df.columns
    ]

    product_mid_price_column = f"{PRODUCT.lower()}_mid_price"
    assert product_mid_price_column in mid_price_df.columns

    trade_history_df = trade_history_df[
        ((trade_history_df["buyer"] == TRADER) | (trade_history_df["seller"] == TRADER))
        & (trade_history_df["symbol"] == PRODUCT)
    ].copy()

    # Computing signed quantity
    trade_history_df["signed_quantity"] = trade_history_df.apply(
        lambda row: (
            row["quantity"]
            if row["buyer"] == TRADER
            else -row["quantity"] if row["seller"] == TRADER else 0
        ),
        axis=1,
    )

    # Creating pivot table
    trade_history_pivot_df = trade_history_df.pivot_table(
        index="timestamp", columns="symbol", values="signed_quantity", aggfunc="sum"
    )

    trade_history_pivot_df.columns = [
        f"{col}_trades_made" for col in trade_history_pivot_df.columns
    ]

    products_trades_made_column = f"{PRODUCT}_trades_made"

    assert products_trades_made_column in trade_history_pivot_df

    normalized_df = mid_price_df[["timestamp", product_mid_price_column]].copy()
    merged_df = pd.merge(normalized_df, trade_history_pivot_df, on="timestamp", how='left')

    # Creating the plot with Plotly
    fig = go.Figure()

    # Add mid price trace
    fig.add_trace(
        go.Scatter(
            x=merged_df["timestamp"],
            y=merged_df[product_mid_price_column],
            mode="lines",
            name="Mid Price",
        )
    )

    # Add trade arrows
    trade_col = f"{PRODUCT}_trades_made"
    if trade_col in merged_df.columns:
        for _, row in merged_df.iterrows():
            if row[trade_col] > 0:  # Buy
                fig.add_trace(
                    go.Scatter(
                        x=[row["timestamp"]],
                        y=[row[product_mid_price_column]],
                        mode="markers",
                        marker_symbol="triangle-up",
                        marker_color="green",
                        marker_size=10,
                        name="Buy",
                    )
                )
            elif row[trade_col] < 0:  # Sell
                fig.add_trace(
                    go.Scatter(
                        x=[row["timestamp"]],
                        y=[row[product_mid_price_column]],
                        mode="markers",
                        marker_symbol="triangle-down",
                        marker_color="red",
                        marker_size=10,
                        name="Sell",
                    )
                )

    fig.update_layout(
        title=f"Time Series of Mid Price and Trades Made for {PRODUCT} by {TRADER}",
        xaxis_title="Timestamp",
        yaxis_title="Price / Trades Made",
        legend_title="Legend",
        font=dict(family="Courier New, monospace", size=12, color="RebeccaPurple"),
    )


    df = trade_history_df
    df_filtered = df[df["symbol"] == PRODUCT]
    # Filter where trader is either buyer or seller
    df_trader = df_filtered[
        (df_filtered["buyer"] == TRADER) | (df_filtered["seller"] == TRADER)
    ]
    df_trader["cum_qty"] = df_trader["signed_quantity"].cumsum()
    df_trader["cash_value"] = (df_trader["price"] * -df_trader["signed_quantity"]).cumsum()
    final_cash_value = df_trader.iloc[-1]["cash_value"]
    final_qty = df_trader.iloc[-1]["cum_qty"]
    # liq_val = df_trader.iloc[-1]["price"]
    liq_val = activities_df[activities_df["product"] == PRODUCT].iloc[-1]["mid_price"]
    net_pnl = final_cash_value + (final_qty * liq_val)
    print(f"{TRADER}: Net P&L for {PRODUCT} = {net_pnl}")

    # Show the figure
    fig.show()

Vladimir: Net P&L for AMETHYSTS = 272.0


Vladimir: Net P&L for CHOCOLATE = 1789.5


Vladimir: Net P&L for COCONUT_COUPON = -13362.0


Vladimir: Net P&L for GIFT_BASKET = -13995.0


Vladimir: Net P&L for ORCHIDS = 3480.0


Vladimir: Net P&L for ROSES = 1862.0


Vladimir: Net P&L for STARFRUIT = 282.0


Vladimir: Net P&L for STRAWBERRIES = -4872.0
